In [ ]:
import sys
PATH_TO_PROJECT = "/home/tanawatp/Documents/Hexapod/PhysicalAI_modelling"
sys.path.append(PATH_TO_PROJECT)
from pydrake.common import temp_directory
from pydrake.geometry import SceneGraphConfig, StartMeshcat
from pydrake.math import RigidTransform, RollPitchYaw
from pydrake.multibody.math import SpatialForce
from pydrake.multibody.parsing import Parser
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.multibody.tree import MultibodyForces
from pydrake.systems.analysis import Simulator
from pydrake.systems.framework import DiagramBuilder, LeafSystem, BasicVector
from pydrake.visualization import AddDefaultVisualization, ModelVisualizer
from pydrake.systems.controllers import PidController
from pydrake.multibody.inverse_kinematics import InverseKinematics
from pydrake.solvers import Solve
from UtilsFunction.UtilsFunc import*
from UtilsFunction.KinematicsFunc import*
from UtilsFunction.HexapodFunc import*
import numpy as np

In [2]:
meshcat = StartMeshcat()
port = str(meshcat.web_url())[-4:]
print(f"tanawatp@tanawatp-ROG-Strix.local:{port}")

INFO:drake:Meshcat listening for connections at http://localhost:7001


tanawatp@tanawatp-ROG-Strix.local:7001


In [ ]:
# Controller configulation
gravity_compensation_enable = True
controller_enable = False
controller = "mit"  # mit, cascade, ID
sim_time_step = 0.0001    # [second]
# Simulation configulation
fixed_base = False
fixed_base_offset = 0.5 # [m] offset for fixed base
gravity_enable = True
realtime = False
save_graph = False
gravity = [0, 0, -9.81*gravity_enable]
sim_time = 0.5 # [second] simulation time
plant_temp,robot_ind = DefinePlant(model_name="Hexapod_mockup",
                    is_fixed=True,
                    fixed_frame="BaseLink")
q_init = 

init_pos = [0.0, 0.0,desire_torso_height+(fixed_base_offset*fixed_base)]
init_orien = np.asarray([0, 0, 0]) # body fram orientation [degree] rotate xyz fixed axis
left_qd = np.zeros(6)
right_qd = np.zeros(6)
robot_urdf = "Hexapod_mockup.urdf"

RuntimeError: /home/tanawatp/Documents/Hexapod/PhysicalAI_modelling/Models/Volta_rev4.urdf:0: error: Failed to parse XML file: XML_ERROR_FILE_NOT_FOUND

In [ ]:
meshcat.Delete()
meshcat.DeleteAddedControls()

dir = "/home/tanawatp/Documents/Hexapod/PhysicalAI_modelling/Models"

robot_urdf_path = os.path.join(dir,robot_urdf)
floor_sdf = "floor_top.sdf"
floor_sdf_path = os.path.join(dir,floor_sdf)

builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(
    builder, time_step=sim_time_step)
parser = Parser(plant)

# Set the gravity feild
plant.mutable_gravity_field().set_gravity_vector(gravity)

# Add models to multibody plant
robot_instant_index = parser.AddModels(robot_urdf_path)
floor = parser.AddModels(floor_sdf_path)

# Weld the floor to the world so that it's fixed during the simulation.
floor_frame = plant.GetFrameByName("floor_top_center")
temp0 = RigidTransform(
    RollPitchYaw([0,0,0]), p=[0.0,0,0])
plant.WeldFrames(plant.world_frame(), floor_frame,temp0)

X_floorRobot = RigidTransform(
    RollPitchYaw(init_orien * np.pi / 180), p=init_pos)
if fixed_base:
    robot_body_frame = plant.GetFrameByName("torso")
    plant.WeldFrames(plant.world_frame(),robot_body_frame, X_floorRobot)
    print("Simulation in Fixed based mode")
else:
    print("Simulation in Folating based mode")

plant.Finalize()

# Set initial pose of robot
plant_context = plant.CreateDefaultContext()
robot = plant.GetBodyByName("BaseLink")
X_Worldfloor = floor_frame.CalcPoseInWorld(plant_context) 

X_WorldRobot = X_Worldfloor.multiply(X_floorRobot)
plant.SetDefaultFreeBodyPose(robot, X_WorldRobot)
